In [1]:
from VoxResNetPytorch import *
import pandas as pd
import numpy as np
import os
import nibabel as nib

import torch.optim as optim
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.autograd import Function
from torch.autograd import gradcheck

import torchvision
import torchvision.transforms as transforms
import torch.utils.data as data
from torchvision.datasets.folder import *



C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def load_nii_to_array(nii_path):
    return nib.load(nii_path).get_data()

In [3]:
def preproc_data(data_file,return_y = True):
    #Encode with 1, psycho, and with 0 normal
    data_file['target'] = 1
    data_file.target[data_file['diagnosis'] == 'CONTROL'] = 0 
    if return_y == True:
        y = participants_pandas[['participant_id','target']]
        return y
    return data_file

In [4]:
class Base_loader_(data.Dataset):
    """ Some documantation"""

    def __init__(self,labels, root_dir, transform=None,loader=default_loader):
        """
        Args:
        """
       
        #First file is targets
        self.root = os.listdir(root_dir)
        print(self.root)
        self.root_path = root_dir

        self.classes = labels.target
        self.class_to_idx = labels.participant_id
        self.transform = transform
        self.loader = loader
        self.size = len(self.root)

    def __getitem__(self, idx):
        target,file_name = self.classes[idx], self.root[idx]
        
        path = os.path.join(self.root_path,file_name)
        
        img = load_nii_to_array(path)
        if self.transform is not None:
            img = self.transform(img)
        return img, target
        
    
    def __len__(self):
        return len(self.root)


In [5]:

data_folder = 'data/sMRI'
targets_file ='data/LA5study_targets.csv' 

participants_pandas = pd.read_csv(targets_file)

y = preproc_data(participants_pandas)
participants_pandas.head()






C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,participant_id,diagnosis,age,gender,bart,bht,dwi,pamenc,pamret,rest,...,T1w,taskswitch,ScannerSerialNumber,ghost_NoGhost,label,Bipolar/Control,Bipolar/NotBipolar,ADHD/Control,Schz/Control,target
0,sub-10159,CONTROL,30,F,1.0,NaN,1.0,NaN,NaN,1.0,...,1.0,1.0,35343.0,No_ghost,CONTROL,0.0,0,0.0,0.0,0
1,sub-10171,CONTROL,24,M,1.0,1.0,1.0,NaN,NaN,1.0,...,1.0,1.0,35343.0,No_ghost,CONTROL,0.0,0,0.0,0.0,0
2,sub-10189,CONTROL,49,M,1.0,NaN,1.0,NaN,NaN,1.0,...,1.0,1.0,35343.0,No_ghost,CONTROL,0.0,0,0.0,0.0,0
3,sub-10193,CONTROL,40,M,1.0,NaN,1.0,NaN,NaN,NaN,...,1.0,NaN,35343.0,No_ghost,CONTROL,0.0,0,0.0,0.0,0
4,sub-10206,CONTROL,21,M,1.0,NaN,1.0,NaN,NaN,1.0,...,1.0,1.0,35343.0,No_ghost,CONTROL,0.0,0,0.0,0.0,0


In [6]:
from torchvision import transforms, utils
tensorize = transforms.ToTensor()
composed = transforms.Compose([tensorize])

loader_train = Base_loader_(y,data_folder,transform = composed)

vox_net = VoxResNet()
vox_net.cuda()


['sub-10159_T1w_space-MNI152NLin2009cAsym_preproc.nii.gz', 'sub-10171_T1w_space-MNI152NLin2009cAsym_preproc.nii.gz', 'sub-10189_T1w_space-MNI152NLin2009cAsym_preproc.nii.gz', 'sub-10193_T1w_space-MNI152NLin2009cAsym_preproc.nii.gz', 'sub-10206_T1w_space-MNI152NLin2009cAsym_preproc.nii.gz', 'sub-10217_T1w_space-MNI152NLin2009cAsym_preproc.nii.gz', 'sub-10225_T1w_space-MNI152NLin2009cAsym_preproc.nii.gz', 'sub-10227_T1w_space-MNI152NLin2009cAsym_preproc.nii.gz', 'sub-10228_T1w_space-MNI152NLin2009cAsym_preproc.nii.gz', 'sub-10235_T1w_space-MNI152NLin2009cAsym_preproc.nii.gz', 'sub-10249_T1w_space-MNI152NLin2009cAsym_preproc.nii.gz', 'sub-10269_T1w_space-MNI152NLin2009cAsym_preproc.nii.gz', 'sub-10271_T1w_space-MNI152NLin2009cAsym_preproc.nii.gz', 'sub-10273_T1w_space-MNI152NLin2009cAsym_preproc.nii.gz', 'sub-10274_T1w_space-MNI152NLin2009cAsym_preproc.nii.gz', 'sub-10280_T1w_space-MNI152NLin2009cAsym_preproc.nii.gz', 'sub-10290_T1w_space-MNI152NLin2009cAsym_preproc.nii.gz', 'sub-10292_T1

C:\ProgramData\Anaconda3\lib\site-packages\torch\cuda\__init__.py:116: UserWarning: 
    Found GPU0 GeForce GTX 670 which is of cuda capability 3.0.
    PyTorch no longer supports this GPU because it is too old.
    
  warnings.warn(old_gpu_warn % (d, name, major, capability[1]))


VoxResNet(
  (model): Sequential(
    (conv3d_1): Conv3d(1, 32, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1))
    (batch_norm_1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (activation_1): ReLU(inplace)
    (conv3d_2): Conv3d(32, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (batch_norm_2): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (activation_2): ReLU(inplace)
    (conv3d_3): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1))
    (block_1): BasicBlock(
      (conv1): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
      (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
      (bn2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_r

In [10]:

epochs_cnt = 10


minibatch_size = 10
test_batch_size = 10

optimizer = torch.optim.SGD(vox_net.parameters(), lr= 0.0001, momentum = 0.9, weight_decay= 0.0001, nesterov= True)
#torch.optim.SGD(params, lr=<object object>, momentum=0, dampening=0, weight_decay=0, nesterov=False)
criterion = nn.CrossEntropyLoss().cuda()

train_loader = torch.utils.data.DataLoader(loader_train, batch_size = minibatch_size, shuffle = True, num_workers = 4)
#test_nc_loader = torch.utils.data.DataLoader(loader, batch_size = test_batch_size, shuffle = False, num_workers = 1)
losses = []
acc_ = []
# примерный код для обучения модели, не обязательно строго следовать ему
for epoch in range(epochs_cnt):
    loss = 0
    train_loss = 0
    for batch_idx, (inputs, labels) in enumerate(train_loader):
        if(batch_idx % 10 == 0):
            print('[%d, %5d] loss: %.3f' % (epoch + 1, batch_idx + 1, train_loss / 10))
            train_loss = 0
        

        inputs, labels = inputs.view(size=(minibatch_size,1,inputs.shape[1],inputs.shape[2],inputs.shape[3])).cuda(), torch.tensor(labels)#labels.view(minibatch_size).cuda()
        optimizer.zero_grad()


        inputs, labels = Variable(inputs), torch.tensor(labels)
       
        outputs = vox_net(inputs)
        print(outputs.shape)
        loss = criterion(outputs, labels)
        train_loss += loss.data[0]
        loss.backward()
        optimizer.step()
    losses.append(loss)

    test_nc2_loader = torch.utils.data.DataLoader(validate_dataset, batch_size = 125, shuffle = False, num_workers = 2)
    acc = acc_check(model_2,test_nc2_loader,minibatch_size)
    acc_.append(acc)
    print(acc)
print('Finished Training')

[1,     1] loss: 0.000


RuntimeError: size mismatch, m1: [1 x 324480], m2: [65536 x 128] at c:\pytorch\aten\src\thc\generic/THCTensorMathBlas.cu:249